In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Colon_and_Rectal_Cancer/GSE108166'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Differential clinicopathological and molecular features within late-onset colorectal cancer according to tumor location"
!Series_summary	"Background: Since there is a predilection of some clinical and molecular features for a given tumor location, we assessed whether this can be confirmed in late-onset colorectal cancer (LOCRC).  Methods: We analyzed clinical and molecular characteristics of LOCRC at different tumor locations in order to determine if there are differential phenotypes related with the location in the colon.  Results: Right colon cancers showed features associated with sporadic Microsatellite Instability: predominance of female cases and BRAF mutations, and an important mucinous component. Left colon cancers developed a higher number of polyps and multiple primary CRCs, showed the strongest familial component, and had better prognosis. Rectal cancers showed a predominantly sporadic phenotype, with worse prognosis and a CpG Island Me

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check the gene expression availability
is_gene_available = True  # Based on background information, assuming gene expression data is available 

# Variable Availability and Data Type Conversion
sample_characteristics = {
    0: ['Sex: Female', 'Sex: Male'],
    1: ['age: 81', 'age: 77', 'age: 85', 'age: 70', 'age: 71', 'age: 75', 'age: 86', 'age: 80', 'age: 87',
        'age: 83', 'age: 93', 'age: 88', 'age: 74', 'age: 78', 'age: 72', 'age: 79', 'age: 76', 'age: 73',
        'age: 69', 'age: 84', 'age: 82', 'age: 89'],
    2: ['molecular classification: MSS y CIMP-High', 'molecular classification: MSS y CIMP-Low/0',
        'molecular classification: MSI y CIMP-Low/0', 'molecular classification: MSI y CIMP-High']
}

# Determine the data availability
trait_row = None  # There is no clear key for 'Colon_and_Rectal_Cancer'
age_row = 1
gender_row = 0

# Define conversion functions
def convert_trait(value):
    return None  # No trait information is available

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except:
        return None

def convert_gender(value):
    try:
        gender = value.split(': ')[1].lower()
        return 1 if gender == 'male' else 0
    except:
        return None

# Save cohort information
save_cohort_info('GSE108166', './preprocessed/Colon_and_Rectal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Colon_and_Rectal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Colon_and_Rectal_Cancer/trait_data/GSE108166.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
